<font size="5" >PURPOSE OF PROJECT: CREATE A REGRESSION MODEL TO PREDICT MONTHLY INCOME </font>

<font size="4"> My project is split into 3 parts. Each notebook focuses on a specific portion of the machine learning workflow. This notebook is part 1 out of 3 of my Regression Model Project. </font>
    
<font size="4"> The focus in this notebook are data cleaning and outlier analysis. See part 2 and part 3 for notebooks on feature selection and model performance evaluation.  </font>

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

# OUTLIER ANALYSIS

The purpose of outlier analysis is to determine if dropping rows with outliers is ideal or not.

CONCLUSION:

1. 47% of rows in the dataset will be dropped. This is too many rows and removing outliers will lead to information loss.
2. The model we are using, Random Forest, is NOT sensitive to outliers.

In [2]:
df = pd.read_csv('ibm_attrition_group_version_1.csv')

In [3]:
'''
obtain all numeric columns for outlier analysis.
'''
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

df_num = df.select_dtypes(include=numerics)

In [4]:
'''
Compute number of rows to be dropped. 

STEPS PERFORMED BY THE ALGORITHM:

1. This code gets the row numbers of outliers in each column. To detect outliers, a boxplot is used.

2. Append the row number of outliers in each column in a 2d array.
    
    For example: [5,7,13,18] append -> [[1,5,7],[3,6,9]] -> [[1,5,7],[3,6,9],[5,7,13,18]]
    
    NOTE: Each list in the 2d array represents the row numbers of outliers i n one column. 
          So if you have n columns, you will have n lists inside the 2d array.

3. To determine the total number of rows to be dropped, a set union is operation is 
   applied to each list inside 2d array. This gets all the unique values in the all nested lists and removes duplicates.

    For Example: [[1,2,3],[1,2,7]] the result after the union operation is {1,2,3,7}
    
4. The set computed in step 3 will contain the unique row numbers which will be dropped. 
   The length of this set is the number of rows to be dropped.

691 rows will be dropped.

47% of rows will be dropped.
'''

row_list = []

for col in df_num.columns:
    
    q3 = df_num[col].quantile(0.75)
    q1 = df_num[col].quantile(0.25)

    iqr = q3 - q1

    col_outliers = df_num[((df_num[col] > (q3 + iqr*1.5)) | (df_num[col] < (q1 - iqr*1.5)))]
    row_list.append(col_outliers['Unnamed: 0'].values)
    
print(len(list(set().union(*row_list))))



691


In [5]:
'''
Calculate the number of outliers for each column

This algorithm filters rows with outliers based on a single column. Then the algorithm computes 
the length of the filtered dataframe, which corresponds to the number of rows. Then the results are stored in
dictionary, where keys are columns names and values, is the number of outliers.
'''

d = {}

for col in df_num.columns:
    
    q3 = df_num[col].quantile(0.75)
    q1 = df_num[col].quantile(0.25)

    iqr = q3 - q1

    col_outliers = df_num[((df_num[col] > (q3 + iqr*1.5)) | (df_num[col] < (q1 - iqr*1.5)))]
    d[col] = len(col_outliers)

d

{'Unnamed: 0': 0,
 'Age': 0,
 'DailyRate': 0,
 'DistanceFromHome': 0,
 'Education': 0,
 'EmployeeNumber': 0,
 'EnvironmentSatisfaction': 0,
 'HourlyRate': 0,
 'JobInvolvement': 0,
 'JobLevel': 0,
 'JobSatisfaction': 0,
 'MonthlyIncome': 114,
 'MonthlyRate': 0,
 'NumCompaniesWorked': 52,
 'PercentSalaryHike': 0,
 'PerformanceRating': 226,
 'RelationshipSatisfaction': 0,
 'StockOptionLevel': 85,
 'TotalWorkingYears': 63,
 'TrainingTimesLastYear': 238,
 'WorkLifeBalance': 0,
 'YearsAtCompany': 104,
 'YearsInCurrentRole': 21,
 'YearsSinceLastPromotion': 107,
 'YearsWithCurrManager': 14}

# GENERAL DATA CLEANING

In [6]:
df = pd.read_csv(r'C:\Users\Angelo Gaerlan\Desktop\Courses\Npower stuff\JDA Npower model project\WA_Fn-UseC_-HR-Employee-Attrition.csv')

In [7]:
'''
Check for constant columns.

If the length of the value_counts() for a specific feature == 1, there is only 1 unique values in that column; therefore, 
the column is non-chnaging.

Non-changing columns will be dropped.

THE FOLLOWING ARE NON-CHANGING COLUMNS:

1. EmployeeCount
2. Over18
3. StandardHours
'''

d = {}

for col in df.columns:
    d[col] = len(df[col].value_counts())

d

{'Age': 43,
 'Attrition': 2,
 'BusinessTravel': 3,
 'DailyRate': 886,
 'Department': 3,
 'DistanceFromHome': 29,
 'Education': 5,
 'EducationField': 6,
 'EmployeeCount': 1,
 'EmployeeNumber': 1470,
 'EnvironmentSatisfaction': 4,
 'Gender': 2,
 'HourlyRate': 71,
 'JobInvolvement': 4,
 'JobLevel': 5,
 'JobRole': 9,
 'JobSatisfaction': 4,
 'MaritalStatus': 3,
 'MonthlyIncome': 1349,
 'MonthlyRate': 1427,
 'NumCompaniesWorked': 10,
 'Over18': 1,
 'OverTime': 2,
 'PercentSalaryHike': 15,
 'PerformanceRating': 2,
 'RelationshipSatisfaction': 4,
 'StandardHours': 1,
 'StockOptionLevel': 4,
 'TotalWorkingYears': 40,
 'TrainingTimesLastYear': 7,
 'WorkLifeBalance': 4,
 'YearsAtCompany': 37,
 'YearsInCurrentRole': 19,
 'YearsSinceLastPromotion': 16,
 'YearsWithCurrManager': 18}

In [8]:
'''
drop all constant columns
'''

df.drop(columns=['EmployeeCount','Over18','StandardHours'],inplace=True)


In [9]:
'''
get all categorical columns
'''

categorical_cols = set(df._get_numeric_data().columns).symmetric_difference(set(df.columns))

categorical_cols

{'Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'OverTime'}

In [10]:
'''
Convert all categorical columns into lower case. 

This ensures that strings can be grouped together with aggregate functions or filtered effectively.
'''

for cat_column in categorical_cols:
    df[cat_column] = df[cat_column].str.lower()

In [11]:
'''
Check if dataframe has missing values.

NOTE: There are no missing values.
'''
df.isnull().sum()

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
dtype: int64

In [12]:
'''
Check if the IBM dataset has any duplicate rows.

Since the dataframe with duplicates dropped has the same length as the orginal dataframe, there are no duplicate rows 
in the dataset
'''
len(df.drop_duplicates()) == len(df)

True

In [13]:
'''
Save the cleaned dataset into a CSV 
'''

df.to_csv('ibm_attrition_group_version_1.csv')